## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-27-10-03-07 +0000,nyt,"In Tuesday’s North Carolina Primaries, the Lef...",https://www.nytimes.com/2026/02/27/us/politics...
1,2026-02-27-10-00-41 +0000,nyt,The Rise and Fall of a 3-D Printing Empire,https://www.nytimes.com/2026/02/27/business/3d...
2,2026-02-27-10-00-35 +0000,nyt,Elon Musk’s Secret Web of Companies in Texas,https://www.nytimes.com/2026/02/27/technology/...
3,2026-02-27-10-00-24 +0000,nyt,"U.S. Birthrate Declines to an All-Time Low, bu...",https://www.nytimes.com/2026/02/27/us/politics...
4,2026-02-27-10-00-18 +0000,nyt,Honolulu on a Budget,https://www.nytimes.com/2026/02/27/travel/hono...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2539/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
391,trump,53
231,epstein,23
114,iran,17
257,ai,17
86,war,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
264,2026-02-26-19-24-19 +0000,nypost,DOJ probes whether any Epstein files about unp...,https://nypost.com/2026/02/26/us-news/doj-prob...,111
142,2026-02-27-00-03-07 +0000,nypost,House Dems to force vote to block Trump from w...,https://nypost.com/2026/02/26/us-news/house-de...,108
136,2026-02-27-00-25-42 +0000,nypost,Trump moves more military might to the Middle ...,https://nypost.com/2026/02/26/us-news/trump-mo...,108
229,2026-02-26-21-28-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...,107
382,2026-02-26-12-32-24 +0000,nypost,"Latest US, Iran talks conclude in Geneva as Tr...",https://nypost.com/2026/02/26/us-news/new-us-i...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
264,111,2026-02-26-19-24-19 +0000,nypost,DOJ probes whether any Epstein files about unp...,https://nypost.com/2026/02/26/us-news/doj-prob...
127,98,2026-02-27-00-53-44 +0000,nypost,Paramount Skydance victory in Warner Bros. Dis...,https://nypost.com/2026/02/26/media/paramount-...
400,76,2026-02-26-11-22-15 +0000,cbc,Iran nuclear talks conclude with no deal annou...,https://www.cbc.ca/news/world/geneva-us-iran-n...
363,66,2026-02-26-14-25-08 +0000,bbc,A New York City snowball fight gone wrong leav...,https://www.bbc.com/news/articles/cwyxlgkj0k3o...
102,62,2026-02-27-02-13-00 +0000,wsj,Hillary Clinton said that she had no knowledge...,https://www.wsj.com/politics/republicans-press...
295,60,2026-02-26-18-08-06 +0000,nypost,Mortgage rates dip below 6% for first time sin...,https://nypost.com/2026/02/26/business/mortgag...
229,51,2026-02-26-21-28-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...
175,46,2026-02-26-22-49-32 +0000,latimes,Downtown LA Law Group moves to block state bar...,https://www.latimes.com/california/story/2026-...
200,45,2026-02-26-22-09-46 +0000,bbc,Burger King rolls out AI headsets that track e...,https://www.bbc.com/news/articles/cgk2zygg0k3o...
20,44,2026-02-27-09-21-00 +0000,wsj,British Airways Owner IAG Posts Higher Net Pro...,https://www.wsj.com/business/earnings/british-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
